In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import re

In [ ]:
result_path = '../tables/cohort_tables'

In [ ]:
with open('../scratch/cohort_stats.pkl', 'rb') as fp:
    data = pickle.load(fp)

In [ ]:
list(data['mimic_inhospital_mortality_target'].index)

In [ ]:
df_long = pd.concat(
    [
        (
            data['mimic_inhospital_mortality_target']
            .rename_axis('group')
            .reset_index()
            .assign(tag = 'mimic')
        ),        
        (
            data['eicu_inhospital_mortality_target']
            .rename_axis('group')
            .reset_index()
            .assign(tag = 'eicu')
        )
    ]
)
df_long

In [ ]:
cohort_statistics_df = (
    df_long
    .rename(columns={'Outcome Incidence': 'In-hospital mortality'})
    .pivot(index='group', columns='tag', values=['In-hospital mortality', 'Count'])
    .reorder_levels([1, 0], axis=1)
    .loc[list(data['mimic_inhospital_mortality_target'].index)]
)
cohort_statistics_df
cohort_statistics_df = cohort_statistics_df[
    [
        ('mimic', 'Count'), 
        ('mimic', 'In-hospital mortality'), 
        ('eicu', 'Count'), 
        ('eicu', 'In-hospital mortality')
    ]
]
cohort_statistics_df

In [ ]:
db_key='mimic_eicu'
table_str = cohort_statistics_df.to_latex(
    buf=None,
#     float_format="%.3g",
    index_names=False,
    index=True,
    label=f"tab:cohort_{db_key}",
    position="!t",
    caption="A caption",
).replace("toprule\n{}", "toprule\n Group")

if isinstance(table_str, tuple):
    table_str = table_str[0]

table_str = re.sub(pattern="\[(?=\d)", repl=r"\\lbrack", string=table_str)

with open(os.path.join(result_path, f"{db_key}.txt"), "w") as fp:
    fp.write(table_str)